<a href="https://colab.research.google.com/github/prxrwx/Simulation/blob/main/Simpy-process.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Install module simpy**

In [ ]:
!pip install simpy

### **import module simpy**

In [ ]:
import simpy

# **Create Processs generator "Car" for simulation**

In [ ]:
# The car will alternately drive and park for a while
def car(env):
  while True:
    print("Start parking at %d" %env.now)
    parking_duration = 5 # can be set to a random value
    yield env.timeout(parking_duration)

    print("Start driving at %d" %env.now)
    trip_duration = 2
    yield env.timeout(trip_duration)

## **Create environment for process and start process car**

In [ ]:
env = simpy.Environment()
env.process(car(env))

<Process(car) object at 0x7fc9f7ab4a90>

## **Run Process with define simulated time**

In [ ]:
# simulation 20 time units
# the car park in 5 time units and drives 2 time units
env.run(until = 20)

Start parking at 0
Start driving at 5
Start parking at 7
Start driving at 12
Start parking at 14
Start driving at 19


#**Process Interaction :- Waiting for a Process**

In [ ]:
class Car(object):
  def __init__(self, env): # constructor method of class Car
    self.env = env
    # start the run process everytime an instance is created
    self.action = env.process(self.run())

  def run(self):
    while True:
      print("Start parking and charging at %d" %self.env.now)
      charge_duration = 6
      # we yield the process that process parking() returns to wait for charge process
      yield self.env.process(self.charge(charge_duration))

      # the charge process has finished and we can start driving again
      print("Start driving at %d" %self.env.now)
      trip_duration = 3
      yield self.env.timeout(trip_duration)

  def charge(self, duration):
    yield self.env.timeout(duration)

env = simpy.Environment() # start process in environment for making model
car = Car(env)
env.run(until = 20)

Start parking and charging at 0
Start driving at 6
Start parking and charging at 9
Start driving at 15
Start parking and charging at 18


#**Process Interaction :- Interrupting Another Process**

In [ ]:
class Car(object):
  def __init__(self, env): # constructor method of class Car
    self.env = env
    # start the run process everytime an instance is created
    self.action = env.process(self.run())

  def run(self):
    while True:
      print("Start parking and charging at %d" %self.env.now)
      charge_duration = 5
      # we may get interrupted while charging thee battery
      try:
        yield self.env.process(self.charge(charge_duration))
      except simpy.Interrupt:
        # when we received an interrupt, we stop charging and switch to the "driving" state
        print("Was interrupted. Fope, the battery is full enough ...")

      print("Start driving at %d" %self.env.now)
      trip_duration = 2
      yield self.env.timeout(trip_duration)

  def charge(self, duration):
    yield self.env.timeout(duration)

def driver(env, car):
  yield env.timeout(3)
  car.action.interrupt()

In [ ]:
env = simpy.Environment()
car = Car(env)
env.process(driver(env, car))

<Process(driver) object at 0x7fc9f79a5c50>

In [ ]:
env.run(until = 30)

Start parking and charging at 0
Was interrupted. Fope, the battery is full enough ...
Start driving at 3
Start parking and charging at 5
Start driving at 10
Start parking and charging at 12
Start driving at 17
Start parking and charging at 19
Start driving at 24
Start parking and charging at 26
